## Import statements for Dataframe manipulation

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Prefence Profile dataframe from CSV file

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Fox - Preference Profiles/Minneapolis21CityCouncilWard2NoWriteins.csv')
data.head(10)

,Num. Voters,rank1,rank2,rank3
0,286,-,-,-
1,483,Cam Gordon,-,-
2,20,Cam Gordon,Guy T Gaskin,-
3,4,Cam Gordon,Guy T Gaskin,Robin Wonsley Worlobah
4,17,Cam Gordon,Guy T Gaskin,Tom Anderson
5,4,Cam Gordon,Guy T Gaskin,Yusra Arab
6,596,Cam Gordon,Robin Wonsley Worlobah,-
7,3,Cam Gordon,Robin Wonsley Worlobah,Guy T Gaskin
8,130,Cam Gordon,Robin Wonsley Worlobah,Tom Anderson
9,334,Cam Gordon,Robin Wonsley Worlobah,Yusra Arab


In [5]:
ranks = data.drop(columns = ['Num. Voters'])
candidates = ranks.stack().unique()
for i in range(candidates.shape[0]):
  if candidates[i] == '-':
    candidates = np.delete(candidates, i)
    break

print(candidates)
print(candidates.shape[0])

['Cam Gordon' 'Guy T Gaskin' 'Robin Wonsley Worlobah' 'Tom Anderson'
 'Yusra Arab']
5


# Count points based on ranking

Determine points at rank with functions

In [6]:
#Allocate points by index in array via borda count
def borda(rank):
  return candidates.shape[0] - rank

#Allocate points via divisible borda count
def divisibleBorda(rank):
  return 2*borda(rank) - 1

#Allocate points via fibonacci borda count
def fibonacci(n):
  return round(1/math.sqrt(5)*pow((1+math.sqrt(5))/2, n))

def fibonacciBorda(rank):
  return fibonacci(borda(rank) + 1)

#Allocate points via exponential borda count
def exponentialBorda(rank):
  return pow(2, borda(rank) - 1)

def quadraticBorda(rank):
  return borda(rank)*(borda(rank) - 1)/2 + 1

# Allocate points

Change data set to numpy matrix

In [8]:
data = data.to_numpy()

Allocate points

In [10]:
points = np.full(len(candidates), 0.0)

useMBC = False #Select true if you want to use MBC

#function to control which counting method is being used and correct
if not useMBC:
  def countVotes(rank):
      return divisibleBorda(rank - 1) #choose counting method

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = candidates.copy()
    lastColumn = 1
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':
        #end loop to begin allocating remaining points
        break
      else:
        #allocate votes based on ranking
        for k in range(candidates.shape[0]):
          if data[i][j] == candidates[k]:
            votedCandidates[k] = 'voted'
            points[k] = points[k] + data[i][0]*countVotes(j)

        lastColumn = j + 1

    # Quick patch
    if countVotes(1) != borda(0):
      #calcualate remaining points
      remainder = 0
      for k in range(lastColumn, candidates.shape[0] + 1):
        remainder = remainder + countVotes(k)

      #allocate remaining points equally among remaining candidates
      if candidates.shape[0] - lastColumn + 1 != 0:
        avg = remainder/(float(candidates.shape[0] - lastColumn + 1))
        for k in range(candidates.shape[0]):
          if candidates[k] == votedCandidates[k]:
            points[k] = points[k] + data[i][0]*avg



MBC Borda

In [28]:
# MBC Method
if useMBC:
  points = np.full(len(candidates), 0.0)

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = []
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':
        break
      else:
        votedCandidates.append(data[i][j])

    for c in range(len(votedCandidates)):
      for k in range(candidates.shape[0]):
        if votedCandidates[c] == candidates[k]:
          points[k] = points[k] + data[i][0]*(len(votedCandidates) - c)

# Check Values

In [11]:
print(candidates)
print(points)
sum = 0
for i in points:
  sum = sum + i

print(sum)

['Cam Gordon' 'Guy T Gaskin' 'Robin Wonsley Worlobah' 'Tom Anderson'
 'Yusra Arab']
[54188. 30893. 53807. 42851. 56086.]
237825.0
